# CEDR : Contextualized Embeddings for Document Ranking

이전 프로젝트로 검색결과 만족도 판별모델과 검색결과 랭킹 모델을 개발해 봤습니다. 이전 프로젝트에서 어려웠던점은 비대칭 데이터,

## Introduction

NLP Task에서 성능을 높히는 가장 일반적인 방법중 하나는 단어의 표현 벡터를 더 잘 구하는 것입니다. 이전의 PACRR, DRMM에서는 Word2Vec, Glove를 사용하였습니다. 논문은 ELMo, BERT와 같이 Contextual을 고려하는 방법을 사용하면 성능 향상을 이루어 낼수 있다고 가설을 세웠습니다.
이에 여러가지 방법을 적용하여 성능향상을 실험적으로 규명하였습니다.

## Methodology



### Notation

기존의 논문들과 같은 표기법을 사용합니다.

### Contextualized similarity tensors

논문에서 강조하는 contextualized representation를 제시하는 파트 입니다. 일반적으로Contextualized language model은 multiple stacked layers를 포함합니다. 직관적으로 layer가 깊을수록 더 많은 컨텍스트가 통합되므로 이를 랭킹모델에 학습시키기 위하여 각 레이어당 유사도를 계산하여 3차원 유사도 표현을 만듭니다.

이에 논문은 contextualized representation을 다음과 같이 정의 합니다.

$$ S_{Q,D}[l, q, d] = cos(context_{Q, D}(q, l), context_{Q, D}(d, l))$$

여기서 $q \in Q, d \in D$는 각 검색어와 검색결과의 단어들입니다.

PACRR과 같은 n-gram based model은 multiple input으로, DRMM과 같은 counting based model은 각 채널에서 갯수를 새어 통합합니다.

###  Joint BERT approach

BERT에서 [CLS] 토큰은 중요한 열할을 합니다.미세조정 과정에서 [CLS] 토큰은 두번째문장을 예측하는 정보를 가지고 있으므로 중요한 feature가 될 수 있습니다.

논문은 이 정보를 활용하기 위하여 검색어 검색어 term에 [CLS] 토큰을 포함하여 유사도를 계산해야 한다고 제안했습니다.



## Experiment

논문에서의 결과 입니다. ELMo와 BERT의 임베딩표현만을 사용한경우에는 성능 향상이 조금 이루어 졌지만 미세조정을 거친 BERT와 [CLS]토큰을 포함한 방법은 상당한 성능 향상을 가져왔습니다. 

제가 직접 joint BERT 방법을 실험해보고 이전 결과와 비교해 보겟습니다.

먼저 BERT모델링을 한후 pretrain과 fine tunning을 직접해보려 하였으나 구현후 pretrain 하는순간 멍청한 생각이란걸 알았습니다... 어떻게 GPU도 없는 노트북에서 BERT를 훈련시킬 생각을 한걸까요? 따라서 구글에서 제공하는 pretrained bert의 weight를 받아서 미세조정을 하기로 하였습니다.

미세 조정은 이전에 했던 검색결과 만족도 판별로 하였습니다. 여기서 한가지 문제가 또 발생하는데, 역시나 imbalance data... 또 저를 괴롭힙니다. 여러 논문들을 찾아본결과 kappa loss, focal loss, eda, weighted cross entropy정도등의 해결책이 있었습니다. 그중 가장 효과가 좋다고 하는 weighted cross entropy로 훈련시킨 결과 kappa score 0.6점대로 머신러닝으로 한달간 개발한 모델과 비슷한 수준으로 나왔습니다.

미세 조정을 마친 후 [CLS]토큰을 포함한 유사도 행렬을 구한후 DRMM, PACRR, DRMM-PACRR에 적용시켜보았습니다.

자세한 코드는 github을 참고해 주세요!